In [66]:
import sqlite3

db_file = 'mydatabase9.db'

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

for table in tables:
    print(table[0])
    


athlete_events
noc_regions


In [63]:
# # removing the duplicate data
# #conn.close()
# cursor.execute('delete from temp_table3')
# conn.commit()

In [67]:
cursor.execute('CREATE TABLE temp_table3 AS SELECT distinct* FROM athlete_events;')
cursor.execute('INSERT into temp_table3 SELECT distinct* FROM temp_table3;')
cursor.execute('DELETE FROM athlete_events;')
cursor.execute('INSERT INTO athlete_events SELECT * FROM temp_table3;')

In [58]:
# conn.commit()

In [68]:
cursor.execute('select * from athlete_events limit 2')
s=cursor.fetchall()
print(s)

[(1, 'A Dijiang', 'M', 24, 180.0, 80.0, 'China', 'CHN', '1992 Summer', 1992, 'Summer', 'Barcelona', 'Basketball', "Basketball Men's Basketball", 'NA'), (2, 'A Lamusi', 'M', 23, 170.0, 60.0, 'China', 'CHN', '2012 Summer', 2012, 'Summer', 'London', 'Judo', "Judo Men's Extra-Lightweight", 'NA')]


In [17]:
# How many olympics games have been held?

In [73]:
cursor.execute('Select count(distinct(games)) from athlete_events')
data = cursor.fetchall()

print(data)

[(51,)]


In [74]:
# List down all Olympics games held so far.

In [75]:
cursor.execute('Select distinct(games) from athlete_events')
data = cursor.fetchall()

print(data)

[('1992 Summer',), ('2012 Summer',), ('1920 Summer',), ('1900 Summer',), ('1988 Winter',), ('1992 Winter',), ('1994 Winter',), ('1932 Summer',), ('2002 Winter',), ('1952 Summer',), ('1980 Winter',), ('2000 Summer',), ('1996 Summer',), ('1912 Summer',), ('1924 Summer',), ('2014 Winter',), ('1948 Summer',), ('1998 Winter',), ('2006 Winter',), ('2008 Summer',), ('2016 Summer',), ('2004 Summer',), ('1960 Winter',), ('1964 Winter',), ('1984 Winter',), ('1984 Summer',), ('1968 Summer',), ('1972 Summer',), ('1988 Summer',), ('1936 Summer',), ('1952 Winter',), ('1956 Winter',), ('1956 Summer',), ('1960 Summer',), ('1928 Summer',), ('1976 Summer',), ('1980 Summer',), ('1964 Summer',), ('2010 Winter',), ('1968 Winter',), ('1906 Summer',), ('1972 Winter',), ('1976 Winter',), ('1924 Winter',), ('1904 Summer',), ('1928 Winter',), ('1908 Summer',), ('1948 Winter',), ('1932 Winter',), ('1936 Winter',), ('1896 Summer',)]


In [76]:
# Mention the total no of nations who participated in each olympics game?

In [78]:
cursor.execute('Select games,count(distinct(noc)) from athlete_events group by year')
data = cursor.fetchall()

print(data)

[('1896 Summer', 12), ('1900 Summer', 31), ('1904 Summer', 15), ('1906 Summer', 21), ('1908 Summer', 22), ('1912 Summer', 29), ('1920 Summer', 29), ('1924 Summer', 46), ('1928 Summer', 46), ('1932 Summer', 47), ('1936 Summer', 50), ('1948 Summer', 61), ('1952 Summer', 69), ('1956 Winter', 75), ('1960 Winter', 84), ('1964 Winter', 94), ('1968 Summer', 112), ('1972 Summer', 121), ('1976 Summer', 93), ('1980 Winter', 90), ('1984 Winter', 148), ('1988 Winter', 160), ('1992 Summer', 170), ('1994 Winter', 67), ('1996 Summer', 197), ('1998 Winter', 72), ('2000 Summer', 200), ('2002 Winter', 77), ('2004 Summer', 201), ('2006 Winter', 79), ('2008 Summer', 204), ('2010 Winter', 82), ('2012 Summer', 205), ('2014 Winter', 89), ('2016 Summer', 207)]


In [79]:
# Which year saw the highest and lowest no of countries participating in olympics

In [103]:
cursor.execute('''
    SELECT year, COUNT(DISTINCT noc)
    FROM athlete_events
    GROUP BY year
    HAVING COUNT(DISTINCT noc) = (
        SELECT COUNT(DISTINCT noc)
        FROM athlete_events
        GROUP BY year
        ORDER BY COUNT(DISTINCT noc) DESC
        LIMIT 1
    ) OR COUNT(DISTINCT noc) = (
        SELECT COUNT(DISTINCT noc)
        FROM athlete_events
        GROUP BY year
        ORDER BY COUNT(DISTINCT noc) ASC
        LIMIT 1
    )
''')

data = cursor.fetchall()
print(data)

[(1896, 12), (2016, 207)]


In [104]:
# Which nation has participated in all of the olympic games

In [140]:
cursor.execute('''with tot_games as
              (select count(distinct games) as total_games
              from athlete_events),
          countries as
              (select games, nr.region as country
              from athlete_events ae
              join noc_regions nr ON nr.noc=ae.noc
              group by games, nr.region),
          countries_participated as
              (select country, count(1) as total_participated_games
              from countries
              group by country)
      select cp.*
      from countries_participated cp
      join tot_games tg on tg.total_games = cp.total_participated_games
      order by 1;
''')
data = cursor.fetchall()

print(data)

[('France', 51), ('Italy', 51), ('Switzerland', 51), ('UK', 51)]


In [141]:
#  Identify the sport which was played in all summer olympics

In [142]:
cursor.execute('''with t1 as
        (select count(distinct games) as total_games
          from athlete_events where season = 'Summer'),
          t2 as
          (select distinct games, sport
          from athlete_events where season = 'Summer'),
          t3 as
          (select sport, count(1) as no_of_games
          from t2
          group by sport)
      select *
      from t3
      join t1 on t1.total_games = t3.no_of_games;
''')
data = cursor.fetchall()

print(data)

[('Athletics', 29, 29), ('Cycling', 29, 29), ('Fencing', 29, 29), ('Gymnastics', 29, 29), ('Swimming', 29, 29)]


In [143]:
#  Which Sports were just played only once in the olympics

In [145]:
cursor.execute('''with t1 as
          (select distinct games, sport
          from athlete_events),
          t2 as
          (select sport, count(1) as no_of_games
          from t1
          group by sport)
      select t2.*, t1.games
      from t2
      join t1 on t1.sport = t2.sport
      where t2.no_of_games = 1
      order by t1.sport
;
''')
data = cursor.fetchall()

print(data)

[('Aeronautics', 1, '1936 Summer'), ('Basque Pelota', 1, '1900 Summer'), ('Cricket', 1, '1900 Summer'), ('Croquet', 1, '1900 Summer'), ('Jeu De Paume', 1, '1908 Summer'), ('Military Ski Patrol', 1, '1924 Winter'), ('Motorboating', 1, '1908 Summer'), ('Racquets', 1, '1908 Summer'), ('Roque', 1, '1904 Summer'), ('Rugby Sevens', 1, '2016 Summer')]


In [146]:
# Fetch the total no of sports played in each olympic games

In [147]:
cursor.execute('''  with t1 as
      (select distinct games, sport
      from athlete_events),
        t2 as
      (select games, count(1) as no_of_sports
      from t1
      group by games)
      select * from t2
      order by no_of_sports desc;
''')
data = cursor.fetchall()

print(data)

[('2000 Summer', 34), ('2004 Summer', 34), ('2008 Summer', 34), ('2016 Summer', 34), ('2012 Summer', 32), ('1996 Summer', 31), ('1992 Summer', 29), ('1988 Summer', 27), ('1920 Summer', 25), ('1984 Summer', 25), ('1908 Summer', 24), ('1936 Summer', 24), ('1972 Summer', 23), ('1976 Summer', 23), ('1980 Summer', 23), ('1964 Summer', 21), ('1900 Summer', 20), ('1924 Summer', 20), ('1948 Summer', 20), ('1968 Summer', 20), ('1952 Summer', 19), ('1956 Summer', 19), ('1960 Summer', 19), ('1904 Summer', 18), ('1932 Summer', 18), ('1912 Summer', 17), ('1928 Summer', 17), ('2002 Winter', 15), ('2006 Winter', 15), ('2010 Winter', 15), ('2014 Winter', 15), ('1998 Winter', 14), ('1906 Summer', 13), ('1992 Winter', 12), ('1994 Winter', 12), ('1924 Winter', 10), ('1964 Winter', 10), ('1968 Winter', 10), ('1972 Winter', 10), ('1976 Winter', 10), ('1980 Winter', 10), ('1984 Winter', 10), ('1988 Winter', 10), ('1896 Summer', 9), ('1948 Winter', 9), ('1928 Winter', 8), ('1936 Winter', 8), ('1952 Winter', 

In [148]:
# Fetch oldest athletes to win a gold medal

In [149]:
cursor.execute('''with temp as
            (select name,sex,cast(case when age = 'NA' then '0' else age end as int) as age
              ,team,games,city,sport, event, medal
            from athlete_events),
        ranking as
            (select *, rank() over(order by age desc) as rnk
            from temp
            where medal='Gold')
    select *
    from ranking
    where rnk = 1
''')
data = cursor.fetchall()

print(data)

[('Charles Jacobus', 'M', 64, 'United States', '1904 Summer', 'St. Louis', 'Roque', "Roque Men's Singles", 'Gold', 1), ('Oscar Gomer Swahn', 'M', 64, 'Sweden', '1912 Summer', 'Stockholm', 'Shooting', "Shooting Men's Running Target, Single Shot, Team", 'Gold', 1), ('Charles Jacobus', 'M', 64, 'United States', '1904 Summer', 'St. Louis', 'Roque', "Roque Men's Singles", 'Gold', 1), ('Oscar Gomer Swahn', 'M', 64, 'Sweden', '1912 Summer', 'Stockholm', 'Shooting', "Shooting Men's Running Target, Single Shot, Team", 'Gold', 1)]
